# 0.0 Imports

In [4]:
import pandas as pd
import numpy  as np
import inflection
from typing import Tuple
from functools import reduce 

# 1.0 Loading Data

In [5]:
%%bigquery df_raw

SELECT * FROM gcpproject-394221.ecommerce_database.ecommerce_table2 WHERE InvoiceDate <= "2023-05-15"

Query is running:   0%|          |

Downloading:   0%|          |

In [6]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,576899,22698,PINK REGENCY TEACUP AND SAUCER,18,2017-11-15,2.95,14911.0,EIRE
1,576899,22699,ROSES REGENCY TEACUP AND SAUCER,18,2017-11-15,2.95,14911.0,EIRE
2,576899,22697,GREEN REGENCY TEACUP AND SAUCER,18,2017-11-15,2.95,14911.0,EIRE
3,576899,22843,BISCUIT TIN VINTAGE GREEN,2,2017-11-15,6.75,14911.0,EIRE
4,576899,22842,BISCUIT TIN VINTAGE RED,2,2017-11-15,6.75,14911.0,EIRE


## 1.1 Rename Columns

In [7]:
# Mudar para snake case
old_cols = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country']

snake_case = lambda x: inflection.underscore(x)

new_cols = list( map(snake_case, old_cols))
df_raw.columns = new_cols

## 1.2 Data Dimensions

In [8]:
print("Número de linhas do conjunto de dados: {}".format(df_raw.shape[0]))
print("Número de colunas do conjunto de dados: {}".format(df_raw.shape[1]))

Número de linhas do conjunto de dados: 404772
Número de colunas do conjunto de dados: 8


## 1.3 Data Types

In [9]:
df_raw.dtypes

invoice_no       object
stock_code       object
description      object
quantity          Int64
invoice_date     dbdate
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4 Change Types

In [10]:
df_raw['invoice_date'] = pd.to_datetime(df_raw['invoice_date'])
df_raw['customer_id'] = df_raw['customer_id'].astype(int)

# 2.0 Data Preparation

In [13]:
def coluna_para_int(dataframe: pd.DataFrame, nome_coluna: str) -> bool:
    """
    Converte a coluna especificada em um dataframe para o tipo inteiro.

    Args:
        dataframe (pd.DataFrame): O dataframe a ser processado.
        nome_coluna (str): O nome da coluna a ser convertida.

    Returns:
        bool: True se a conversão foi bem sucedida, False caso contrário.
    """
    
    if nome_coluna not in dataframe.columns:
        raise ValueError(f'Coluna {nome_coluna} nao encontrada')
        
    try:
        dataframe[nome_coluna] = dataframe[nome_coluna].astype(int)
    except:
        raise ValueError(f'Nao foi possivel converter a coluna {nome_coluna} para inteiro.')
    return True

In [16]:
coluna_para_int(df_raw, 'customer_id')

True

In [17]:
def coluna_para_date(dataframe: pd.DataFrame, nome_coluna: str, formato_data: str = None) -> bool: 
    """
    Descricao
    
    Args:
    
    Retornos:
    """
    
    if nome_coluna not in dataframe.columns:
        raise ValueError(f'Coluna {nome_coluna} nao foi encontrada no dataframe de entrada')

    try:
        if formato_data:
            dataframe[nome_coluna] = pd.to_datetime(dataframe[nome_coluna], format=formato_data)
        else:
            dataframe[nome_coluna] = pd.to_datetime(dataframe[nome_coluna])
    except (ValueError, TypeError):
        raise ValueError(f"Could not convert column '{nome_coluna}'")

    # Retorna True se a conversão foi bem sucedida
    return True

In [27]:
def keep_features(dataframe: pd.DataFrame, keep_columns: list) -> pd.DataFrame:
    """
    Retorna um DataFrame com as colunas especificadas em keep_columns.
    
    Args:
        dataframe (pd.DataFrame): O DataFrame a ser processado.
        keep_columns (list): A lista de nomes de colunas a serem mantidas no DataFrame resultante.

    Returns:
        pd.DataFrame: O DataFrame resultante com apenas as colunas especificadas em keep_columns.
    """
    return dataframe[keep_columns]

def filtragem_features(dataframe_raw: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Descricao
    
    Args:
    
    Retornos:
    """
    df_returns = dataframe_raw.loc[dataframe_raw['quantity'] < 0, ['customer_id', 
                                                                   'quantity']]
    df_purchases = dataframe_raw.loc[dataframe_raw['quantity'] >= 0, :]
    
    # Filter main data
    df_filtered = keep_features(dataframe_raw, ['invoice_no', 'stock_code', 'quantity',
                                                'invoice_date', 'unit_price', 
                                                'customer_id', 'country'])
    
    return df_filtered, df_purchases, df_returns

In [28]:
df_raw.columns

Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country'],
      dtype='object')

In [29]:
df_filtered, df_purchases, df_returns = filtragem_features(df_raw)

In [30]:
df_filtered.head(1)

,invoice_no,stock_code,quantity,invoice_date,unit_price,customer_id,country
0,576899,22698,18,2017-11-15,2.95,14911,EIRE


In [31]:
df_purchases.head(1)

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,576899,22698,PINK REGENCY TEACUP AND SAUCER,18,2017-11-15,2.95,14911,EIRE


In [32]:
df_returns.head(1)

,customer_id,quantity
191,12670,-5


# 3.0. Feature engineering

In [11]:
def calculate_gross_revenue(dataframe_purchases: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula a receita bruta de cada cliente com base nas colunas 'Quantity' e 'UnitPrice' e retorna
    um DataFrame com as colunas 'CustomerID' e 'gross_revenue'.

    Args:
        dataframe_purchases (pd.DataFrame): O DataFrame das compras contendo as colunas 'CustomerID', 'Quantity' e 'UnitPrice'.

    Returns:
        pd.DataFrame: O DataFrame resultante contendo as colunas 'CustomerID' e 'gross_revenue'.
    """
    # Verifica se as colunas necessárias estão presentes no DataFrame de entrada
    required_columns = {'customer_id', 'quantity', 'unit_price'}
    missing_columns = required_columns - set(dataframe_purchases.columns)
    if missing_columns:
        raise ValueError(f"O DataFrame de entrada está faltando as seguintes colunas: {missing_columns}")
    
    # Calcula a receita bruta e agrupa por CustomerID
    df = dataframe_purchases.copy()
    df.loc[:, 'gross_revenue'] = df.loc[:, 'quantity'] * df.loc[:, 'unit_price']
    grouped_df = df.groupby('customer_id').agg({'gross_revenue': 'sum'}).reset_index().copy()
    
    return grouped_df

def create_recency(dataframe_purchases: pd.DataFrame, dataframe_filtered: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula a recência da última compra para cada cliente.

    Args:
        dataframe_purchases (pd.DataFrame): DataFrame com as informações de compras de todos os clientes.
        dataframe_filtered (pd.DataFrame): DataFrame filtrado apenas com as informações dos clientes que desejamos calcular a recência.

    Returns:
        pd.DataFrame: DataFrame com as colunas 'CustomerID' e 'recency_days', indicando a recência em dias da última compra para cada cliente.

    """
    required_columns = {'customer_id', 'invoice_date'}
    missing_columns = required_columns - set(dataframe_purchases.columns)
    if missing_columns:
        raise ValueError(f"O DataFrame de entrada está faltando as seguintes colunas: {missing_columns}")
    # calcula a data da última compra de cada cliente
    df_recency = dataframe_purchases.loc[:, ['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
    
    # calcula a recência em dias da última compra de cada cliente em relação à data mais recente da base de dados filtrada
    df_recency['recency_days'] = (dataframe_filtered['invoice_date'].max() - df_recency['invoice_date']).dt.days
    
    # retorna o DataFrame apenas com as colunas 'CustomerID' e 'recency_days'
    return df_recency[['customer_id', 'recency_days']]

def create_quantity_purchased(dataframe_purchases: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula a quantidade de produtos adquiridos por cada cliente.

    Args:
        dataframe_purchases (pd.DataFrame): DataFrame com as informações de compras de todos os clientes.

    Returns:
        pd.DataFrame: DataFrame com as colunas 'CustomerID' e 'qty_products', indicando a quantidade de produtos adquiridos por cada cliente.
    """
    required_columns = {'customer_id', 'stock_code'}
    missing_columns = required_columns - set(dataframe_purchases.columns)
    if missing_columns:
        raise ValueError(f"O DataFrame de entrada está faltando as seguintes colunas: {missing_columns}")
    # agrupa as informações de compras por CustomerID e conta o número de StockCode para cada grupo
    qty_purchased = dataframe_purchases.loc[:, ['customer_id', 'stock_code']].groupby('customer_id').count()
    
    # renomeia a coluna StockCode para qty_products e reseta o índice para transformar o CustomerID em uma coluna
    qty_purchased = qty_purchased.reset_index().rename(columns={'stock_code': 'qty_products'})
    
    # retorna o DataFrame com as colunas 'CustomerID' e 'qty_products'
    return qty_purchased
def create_freq_purchases(dataframe_purchases: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the purchase frequency of each customer based on the purchase history.

    Parameters
    ----------
    dataframe_purchases : pd.DataFrame
        DataFrame with purchase history of each customer, containing columns CustomerID, InvoiceNo, and InvoiceDate.

    Returns
    -------
    pd.DataFrame
        DataFrame with the purchase frequency of each customer, containing columns CustomerID and frequency.
    """
    required_columns = {'customer_id', 'invoice_no', 'invoice_date'}
    missing_columns = required_columns - set(dataframe_purchases.columns)
    if missing_columns:
        raise ValueError(f"O DataFrame de entrada está faltando as seguintes colunas: {missing_columns}")

    # Calculate time range of purchases for each customer
    df_aux = (dataframe_purchases[['customer_id', 'invoice_no', 'invoice_date']]
              .drop_duplicates()
              .groupby('customer_id')
              .agg(max_=('invoice_date', 'max'),
                   min_=('invoice_date', 'min'),
                   days_=('invoice_date', lambda x: ((x.max() - x.min()).days) + 1),
                   buy_=('invoice_no', 'count'))
              .reset_index())

    # Calculate frequency of purchases for each customer
    df_aux['frequency'] = df_aux[['buy_', 'days_']].apply(
        lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis=1)

    return df_aux
def create_qty_returns(dataframe_returns: pd.DataFrame) -> pd.DataFrame:
    """
    Computes the total quantity of returned products for each customer.

    Args:
        dataframe_returns: A pandas DataFrame containing information about returns.

    Returns:
        A pandas DataFrame with the total quantity of returned products for each customer.
    """
    # Validate input data
    
    if not all(col in dataframe_returns.columns for col in ['customer_id', 'quantity']):
        raise ValueError("Input DataFrame must contain 'CustomerID' and 'Quantity' columns")
    
    # Compute quantity of returns
    df_returns = dataframe_returns[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 'qty_returns'})
    df_returns['qty_returns'] = df_returns['qty_returns']* -1
    
    return df_returns
def run_feature_engineering(dataframe_filtered: pd.DataFrame, dataframe_purchases: pd.DataFrame, dataframe_returns: pd.DataFrame) -> pd.DataFrame:
    """
    Performs feature engineering on the input dataframes and returns a new dataframe with the engineered features.

    Args:
        dataframe_filtered: A pandas DataFrame containing filtered customer order data.
        dataframe_purchases: A pandas DataFrame containing customer purchase data.
        dataframe_returns: A pandas DataFrame containing customer return data.

    Returns:
        A pandas DataFrame with the engineered features for each customer.
    """
    # Check if input dataframes are empty
    if dataframe_filtered.empty:
        raise ValueError("Input DataFrame 'dataframe_filtered' is empty")
    if dataframe_purchases.empty:
        raise ValueError("Input DataFrame 'dataframe_purchases' is empty")

    # Check if required columns are present in input dataframes
    required_columns = ['customer_id', 'invoiceDate', 'stock_code', 'quantity', 'unit_price']
    for df, name in zip([dataframe_filtered, dataframe_purchases], ['dataframe_filtered', 'dataframe_purchases']):
        missing_columns = set(required_columns) - set(df.columns)
        if missing_columns:
            raise ValueError(f"Missing columns {missing_columns} in input DataFrame '{name}'")
    if 'customer_id' not in dataframe_returns.columns:
        raise ValueError("Column 'CustomerID' not found in input DataFrame 'dataframe_returns'")
    if 'quantity' not in dataframe_returns.columns:
        raise ValueError("Column 'Quantity' not found in input DataFrame 'dataframe_returns'")

    # Perform feature engineering
    df_fengi = keep_features(dataframe_filtered, ['CustomerID']).drop_duplicates(ignore_index=True)
    gross_revenue = calculate_gross_revenue(dataframe_purchases)
    df_recency = create_recency(dataframe_purchases, dataframe_filtered)
    df_qty_products = create_quantity_purchased(dataframe_purchases)
    df_freq = create_freq_purchases(dataframe_purchases)
    returns = create_qty_returns(dataframe_returns)
    
    # Merge dataframes
    dfs = [df_fengi, gross_revenue, df_recency, df_qty_products, df_freq, returns]
    df_fengi = reduce(lambda left,right: pd.merge(left, right, on='CustomerID', how='left'), dfs)

    # Fill NaN values
    df_fengi['qty_returns'] = df_fengi['qty_returns'].fillna(0)

    # Select final features and return dataframe
    features = ['CustomerID', 'gross_revenue', 'recency_days', 'qty_products', 'frequency', 'qty_returns']
    return keep_features(df_fengi, features).dropna()

In [12]:
df_fengi = run_feature_engineering(df_filtered, df_purchases, df_returns)
df_fengi.head()

NameError: name 'df_filtered' is not defined